In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.orm import sessionmaker, declarative_base

# No need for RunnableConfig or relationship in this schema file
# from langchain_core.runnables.config import RunnableConfig
# from sqlalchemy.orm import relationship

load_dotenv()

# --- IMPORTANT: Updated to point to your new database ---
DATABASE_URL = os.getenv("DATABASE_URL", "sqlite:///netcdf_database.db")
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()


# Definition of the Ocean Data table
# This class maps directly to the 'ocean_data' table created from your NetCDF file.
class OceanData(Base):
    __tablename__ = "ocean_data"

    # It's good practice to have a primary key, even if not in the original file
    id = Column(Integer, primary_key=True, index=True)
    
    # Coordinates from the NetCDF file
    time = Column(DateTime)
    latitude = Column(Float)
    longitude = Column(Float)
    depth = Column(Float)
    
    # Variables from the NetCDF file
    so = Column(Float)       # Sea Water Salinity
    thetao = Column(Float)   # Sea Water Potential Temperature
    uo = Column(Float)       # Eastward Sea Water Velocity
    vo = Column(Float)       # Northward Sea Water Velocity
    zos = Column(Float)      # Sea Surface Height Above Geoid

# Example of how to create the table if needed (the setup_db.py already did this)
# if __name__ == "__main__":
#     print("Creating database tables...")
#     Base.metadata.create_all(bind=engine)
#     print("Tables created.")

In [3]:
import os
from dotenv import load_dotenv
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from sqlalchemy import text, inspect
from langgraph.graph import StateGraph, END
# Assuming your schema definition is in a file named db_schema.py
from db_schema import engine, SessionLocal

# --- This connects to the Ollama application you installed ---
llm = ChatOllama(model="phi3", temperature=0)

# --- UPDATED: Removed 'current_user' as it's no longer relevant ---
class AgentState(TypedDict):
    question: str
    sql_query: str
    query_result: str
    query_rows: list
    attempts: int
    relevance: str
    sql_error: bool

def get_database_schema(engine):
    """Inspects the database and returns its schema."""
    inspector = inspect(engine)
    schema = ""
    for table_name in inspector.get_table_names():
        schema += f"Table: {table_name}\n"
        for column in inspector.get_columns(table_name):
            col_name = column["name"]
            col_type = str(column["type"])
            schema += f"- {col_name}: {col_type}\n"
        schema += "\n"
    print("Retrieved database schema.")
    return schema

# --- REMOVED: The get_current_user function is no longer needed ---

class CheckRelevance(BaseModel):
    relevance: str = Field(
        description="Indicates whether the question is related to the database schema. 'relevant' or 'not_relevant'."
    )

def check_relevance(state: AgentState):
    """Checks if the user's question is relevant to the ocean data schema."""
    question = state["question"]
    schema = get_database_schema(engine)
    print(f"Checking relevance of the question: {question}")
    
    # --- UPDATED: Prompt is now specific to oceanographic data ---
    system = """You are an assistant that determines if a question is related to the database schema of oceanographic data.
    A question is relevant if it asks about sea temperature, salinity, velocity, coordinates, depth, or time.

    For example:
    - "What is the sea water temperature at latitude 45 and longitude 10?" is relevant.
    - "Find the salinity at a depth of 50 meters." is relevant.
    - "Who directed the movie Inception?" is not_relevant.

    Schema:
    {schema}

    Respond with a JSON object with a single key 'relevance' and a value of 'relevant' or 'not_relevant'.
    """.format(schema=schema)
    
    human = f"Question: {question}"
    check_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
    
    parser = PydanticOutputParser(pydantic_object=CheckRelevance)
    json_llm = llm.bind(format="json") # Simplified binding for phi3
    relevance_checker = check_prompt | json_llm | parser
    
    relevance = relevance_checker.invoke({"question": question})
    state["relevance"] = relevance.relevance
    print(f"Relevance determined: {state['relevance']}")
    return state

class ConvertToSQL(BaseModel):
    sql_query: str = Field(description="The SQL query for the user's question.")

def convert_nl_to_sql(state: AgentState):
    """Converts the natural language question to an SQL query."""
    question = state["question"]
    schema = get_database_schema(engine)
    print(f"Converting question to SQL: {question}")

    # --- UPDATED: Prompt is now geared towards the ocean_data table ---
    system = """You are an expert SQL assistant. Convert natural language questions into SQL queries based on the provided schema.
    The user is asking about oceanographic data. The table is named 'ocean_data'.
    Pay close attention to column names: 'latitude', 'longitude', 'depth', 'time', 'so' (salinity), 'thetao' (temperature), 'uo' (eastward velocity), 'vo' (northward velocity), 'zos' (sea surface height).
    When querying for a specific coordinate, find the closest available data point.

    Schema:
    {schema}
    
    Respond with a JSON object with a single key 'sql_query' containing only the SQL query.
    """.format(schema=schema)
    
    convert_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
    
    parser = PydanticOutputParser(pydantic_object=ConvertToSQL)
    json_llm = llm.bind(format="json")
    sql_generator = convert_prompt | json_llm | parser
    
    result = sql_generator.invoke({"question": question})
    state["sql_query"] = result.sql_query
    print(f"Generated SQL query: {state['sql_query']}")
    return state

def execute_sql(state: AgentState):
    """Executes the SQL query and returns the result."""
    sql_query = state["sql_query"].strip()
    session = SessionLocal()
    print(f"Executing SQL query: {sql_query}")
    try:
        result = session.execute(text(sql_query))
        if sql_query.lower().strip().startswith("select"):
            rows = result.fetchall()
            columns = result.keys()
            state["query_rows"] = [dict(zip(columns, row)) for row in rows]
            # --- UPDATED: Simplified result, passing raw rows to the next step ---
            state["query_result"] = f"Query returned {len(state['query_rows'])} rows."
            print(f"Raw SQL Query Result: {state['query_rows']}")
        else:
            session.commit()
            state["query_result"] = "The action has been successfully completed."
        state["sql_error"] = False
    except Exception as e:
        state["query_result"] = f"Error executing SQL query: {str(e)}"
        state["sql_error"] = True
        print(f"Error executing SQL query: {str(e)}")
    finally:
        session.close()
    return state

def generate_human_readable_answer(state: AgentState):
    """Generates a clear, natural language response from the query results."""
    question = state["question"]
    query_rows = state.get("query_rows", [])
    sql_error = state.get("sql_error", False)
    print("Generating a human-readable answer.")

    # --- UPDATED: Prompts are now for scientific data ---
    system = "You are an assistant who explains database query results about oceanographic data in a clear and concise way."
    
    if sql_error:
        prompt_text = f"The query for the question '{question}' failed. Explain the error in a simple sentence: {state['query_result']}"
    elif not query_rows:
        prompt_text = f"Based on the question '{question}', formulate a single sentence stating that no data was found for the specified criteria."
    else:
        prompt_text = f"""The user asked: '{question}'.
        The database returned the following data: {query_rows}.
        
        Formulate a clear, natural language answer summarizing this data in a single sentence.
        For example: 'At the specified coordinates, the sea water temperature (thetao) is X degrees and the salinity (so) is Y.'
        """
        
    generate_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", prompt_text)])
    
    human_response = generate_prompt | llm | StrOutputParser()
    answer = human_response.invoke({})
    state["query_result"] = answer
    print("Generated human-readable answer.")
    return state

# --- The regeneration and funny response nodes remain largely the same, but will be called less often ---
class RewrittenQuestion(BaseModel):
    question: str = Field(description="The rewritten question.")

def regenerate_query(state: AgentState):
    question = state["question"]
    print("Regenerating the SQL query by rewriting the question.")
    system = """You are an assistant that reformulates a question about oceanographic data to be more precise for SQL conversion.
    Ensure that details like latitude, longitude, and depth are clearly stated.
    Respond with a JSON object with a single key 'question' containing the rewritten question."""
    rewrite_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", f"Original Question: {question}")])
    
    parser = PydanticOutputParser(pydantic_object=RewrittenQuestion)
    json_llm = llm.bind(format="json")
    rewriter = rewrite_prompt | json_llm | parser
    
    rewritten = rewriter.invoke({})
    state["question"] = rewritten.question
    state["attempts"] += 1
    print(f"Rewritten question: {state['question']}")
    return state

# ... (generate_funny_response, end_max_iterations, and routers can remain as they are)
def generate_funny_response(state: AgentState):
    print("Generating a funny response for an unrelated question.")
    system = """You are a charming and funny assistant who responds in a playful manner when asked an irrelevant question."""
    human_message = "I am an expert on ocean data. While that is a fascinating question, it is a bit outside my area of expertise. How about we check the sea surface temperature somewhere?"
    funny_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human_message)])
    funny_llm = ChatOllama(model="phi3", temperature=0.7)
    funny_response = funny_prompt | funny_llm | StrOutputParser()
    message = funny_response.invoke({})
    state["query_result"] = message
    print("Generated funny response.")
    return state

def end_max_iterations(state: AgentState):
    state["query_result"] = "I am having trouble formulating a query for your request. Please try rephrasing."
    print("Maximum attempts reached. Ending the workflow.")
    return state

def relevance_router(state: AgentState):
    if state["relevance"].lower() == "relevant":
        return "convert_to_sql"
    else:
        return "generate_funny_response"

def check_attempts_router(state: AgentState):
    if state["attempts"] < 3:
        return "convert_to_sql"
    else:
        return "end_max_iterations"

def execute_sql_router(state: AgentState):
    if not state.get("sql_error", False):
        return "generate_human_readable_answer"
    else:
        return "regenerate_query"

# --- UPDATED: The graph workflow is simplified ---
workflow = StateGraph(AgentState)
workflow.add_node("check_relevance", check_relevance)
workflow.add_node("convert_to_sql", convert_nl_to_sql)
workflow.add_node("execute_sql", execute_sql)
workflow.add_node("generate_human_readable_answer", generate_human_readable_answer)
workflow.add_node("regenerate_query", regenerate_query)
workflow.add_node("generate_funny_response", generate_funny_response)
workflow.add_node("end_max_iterations", end_max_iterations)

# --- The entry point is now check_relevance ---
workflow.set_entry_point("check_relevance")

workflow.add_conditional_edges(
    "check_relevance",
    relevance_router,
    {"convert_to_sql": "convert_to_sql", "generate_funny_response": "generate_funny_response"},
)
workflow.add_edge("convert_to_sql", "execute_sql")
workflow.add_conditional_edges(
    "execute_sql",
    execute_sql_router,
    {"generate_human_readable_answer": "generate_human_readable_answer", "regenerate_query": "regenerate_query"},
)
workflow.add_conditional_edges(
    "regenerate_query",
    check_attempts_router,
    {"convert_to_sql": "convert_to_sql", "end_max_iterations": "end_max_iterations"},
)
workflow.add_edge("generate_human_readable_answer", END)
workflow.add_edge("generate_funny_response", END)
workflow.add_edge("end_max_iterations", END)

app = workflow.compile()

# --- Example of how to run the agent ---
if __name__ == "__main__":
    print("SQL Agent for Ocean Data is ready. Ask a question.")
    
    # Example question
    user_question = "what is the sea water potential temperature and salinity at longitude -10 and latitude 50 at a depth of 5 meters?"
    
    # The config is no longer needed for user_id
    inputs = {"question": user_question, "attempts": 0}
    
    for event in app.stream(inputs):
        for key, value in event.items():
            print(f"--- Event: {key} ---")
            print(value)
            print("\n")

C:\Users\apran\AppData\Local\Temp\ipykernel_22912\2072648219.py:14: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="phi3", temperature=0)


SQL Agent for Ocean Data is ready. Ask a question.
Retrieved database schema.
Checking relevance of the question: what is the sea water potential temperature and salinity at longitude -10 and latitude 50 at a depth of 5 meters?
Relevance determined: relevant
--- Event: check_relevance ---
{'question': 'what is the sea water potential temperature and salinity at longitude -10 and latitude 50 at a depth of 5 meters?', 'attempts': 0, 'relevance': 'relevant'}


Retrieved database schema.
Converting question to SQL: what is the sea water potential temperature and salinity at longitude -10 and latitude 50 at a depth of 5 meters?
Generated SQL query: SELECT time, so, thetao FROM ocean_data WHERE (latitude = 50 AND longitude = -10) AND depth = 5 ORDER BY ABS(time - CURRENT_TIMESTAMP), latitude ASC LIMIT 1;
--- Event: convert_to_sql ---
{'question': 'what is the sea water potential temperature and salinity at longitude -10 and latitude 50 at a depth of 5 meters?', 'attempts': 0, 'relevance': 'r

In [7]:
import os
from dotenv import load_dotenv
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from sqlalchemy import text, inspect
from langgraph.graph import StateGraph, END
# Assuming your schema definition is in a file named db_schema.py
from db_schema import engine, SessionLocal

# --- This connects to the Ollama application you installed ---
llm = ChatOllama(model="phi3", temperature=0)

class AgentState(TypedDict):
    question: str
    sql_query: str
    query_result: str
    query_rows: list
    attempts: int
    relevance: str
    sql_error: bool

def get_database_schema(engine):
    """Inspects the database and returns its schema."""
    inspector = inspect(engine)
    schema = ""
    for table_name in inspector.get_table_names():
        schema += f"Table: {table_name}\n"
        for column in inspector.get_columns(table_name):
            col_name = column["name"]
            col_type = str(column["type"])
            schema += f"- {col_name}: {col_type}\n"
        schema += "\n"
    print("Retrieved database schema.")
    return schema

class CheckRelevance(BaseModel):
    relevance: str = Field(
        description="Indicates whether the question is related to the database schema. 'relevant' or 'not_relevant'."
    )

def check_relevance(state: AgentState):
    """Checks if the user's question is relevant to the ocean data schema."""
    question = state["question"]
    schema = get_database_schema(engine)
    print(f"Checking relevance of the question: {question}")
    
    system = """You are an assistant that determines if a question is related to the database schema of oceanographic data.
    A question is relevant if it asks about sea temperature, salinity, velocity, coordinates, depth, or time.

    For example:
    - "What is the sea water temperature at latitude 45 and longitude 10?" is relevant.
    - "Find the salinity at a depth of 50 meters." is relevant.
    - "Who directed the movie Inception?" is not_relevant.

    Schema:
    {schema}

    Respond with a JSON object with a single key 'relevance' and a value of 'relevant' or 'not_relevant'.
    """.format(schema=schema)
    
    # --- FIX: Use a placeholder for the question ---
    check_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "Question: {question}")])
    
    parser = PydanticOutputParser(pydantic_object=CheckRelevance)
    json_llm = llm.bind(format="json")
    relevance_checker = check_prompt | json_llm | parser
    
    relevance = relevance_checker.invoke({"question": question})
    state["relevance"] = relevance.relevance
    print(f"Relevance determined: {state['relevance']}")
    return state

class ConvertToSQL(BaseModel):
    sql_query: str = Field(description="The SQL query for the user's question.")

def convert_nl_to_sql(state: AgentState):
    """Converts the natural language question to a robust SQL query."""
    question = state["question"]
    schema = get_database_schema(engine)
    print(f"Converting question to SQL: {question}")

    system = """You are an expert SQL assistant. Convert natural language questions into robust SQL queries based on the provided schema.
    The user is asking about oceanographic data in the 'ocean_data' table.
    
    IMPORTANT: The data exists on a grid. Do not use exact equality checks (e.g., WHERE latitude = 50).
    Instead, find the NEAREST available data point by using an ORDER BY clause with a squared distance calculation and then taking the first result (LIMIT 1).

    Here is a full example. For a question asking about latitude 50, longitude -10, and depth 5, the correct query is:
    'SELECT * FROM ocean_data ORDER BY (latitude - 50.0)*(latitude - 50.0) + (longitude - -10.0)*(longitude - -10.0) + (depth - 5.0)*(depth - 5.0) ASC LIMIT 1;'
    
    Pay close attention to column names: 'latitude', 'longitude', 'depth', 'time', 'so' (salinity), 'thetao' (temperature).

    Schema:
    {schema}
    
    Respond with a JSON object with a single key 'sql_query' containing only the SQL query.
    """.format(schema=schema)
    
    convert_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
    
    parser = PydanticOutputParser(pydantic_object=ConvertToSQL)
    json_llm = llm.bind(format="json")
    sql_generator = convert_prompt | json_llm | parser
    
    result = sql_generator.invoke({"question": question})
    state["sql_query"] = result.sql_query
    print(f"Generated SQL query: {state['sql_query']}")
    return state

def execute_sql(state: AgentState):
    """Executes the SQL query and returns the result."""
    sql_query = state["sql_query"].strip()
    session = SessionLocal()
    print(f"Executing SQL query: {sql_query}")
    try:
        result = session.execute(text(sql_query))
        if sql_query.lower().strip().startswith("select"):
            rows = result.fetchall()
            columns = result.keys()
            state["query_rows"] = [dict(zip(columns, row)) for row in rows]
            state["query_result"] = f"Query returned {len(state['query_rows'])} rows."
            print(f"Raw SQL Query Result: {state['query_rows']}")
        else:
            session.commit()
            state["query_result"] = "The action has been successfully completed."
        state["sql_error"] = False
    except Exception as e:
        state["query_result"] = f"Error executing SQL query: {str(e)}"
        state["sql_error"] = True
        print(f"Error executing SQL query: {str(e)}")
    finally:
        session.close()
    return state

def generate_human_readable_answer(state: AgentState):
    """Generates a clear, natural language response from the query results."""
    question = state["question"]
    query_rows = state.get("query_rows", [])
    sql_error = state.get("sql_error", False)
    print("Generating a human-readable answer.")

    system = "You are an assistant who explains database query results about oceanographic data in a clear and concise way."
    
    # --- KEY FIX: The logic for creating the prompt text and invoking the model is updated ---
    prompt_text = ""
    if sql_error:
        prompt_text = f"The query for the question '{question}' failed. Explain the error in a simple sentence: {state['query_result']}"
    elif not query_rows:
        prompt_text = f"Based on the question '{question}', formulate a single sentence stating that no data was found for the specified criteria."
    else:
        # Format the data cleanly to avoid raw dicts with curly braces in the prompt
        data_summary = ", ".join([str(row) for row in query_rows])
        prompt_text = f"""The user asked: '{question}'.
        The database returned the following data: {data_summary}.
        
        Formulate a clear, natural language answer summarizing this data in a single sentence. Mention the values and the coordinates found.
        For example: 'At the nearest point (latitude X, longitude Y, depth Z), the sea water temperature (thetao) is A degrees and the salinity (so) is B.'
        """
    
    # Use a placeholder in the template
    generate_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input_text}")])
    human_response = generate_prompt | llm | StrOutputParser()
    # Provide the formatted text as a variable
    answer = human_response.invoke({"input_text": prompt_text})
    
    state["query_result"] = answer
    print("Generated human-readable answer.")
    return state

class RewrittenQuestion(BaseModel):
    question: str = Field(description="The rewritten question.")

def regenerate_query(state: AgentState):
    question = state["question"]
    print("Regenerating the SQL query by rewriting the question.")
    system = """You are an assistant that reformulates a question about oceanographic data to be more precise for SQL conversion.
    Ensure that details like latitude, longitude, and depth are clearly stated.
    Respond with a JSON object with a single key 'question' containing the rewritten question."""
    
    # --- FIX: Use a placeholder for the question ---
    rewrite_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "Original Question: {question}")])
    
    parser = PydanticOutputParser(pydantic_object=RewrittenQuestion)
    json_llm = llm.bind(format="json")
    rewriter = rewrite_prompt | json_llm | parser
    
    rewritten = rewriter.invoke({"question": question})
    state["question"] = rewritten.question
    state["attempts"] += 1
    print(f"Rewritten question: {state['question']}")
    return state

def generate_funny_response(state: AgentState):
    print("Generating a funny response for an unrelated question.")
    system = """You are a charming and funny assistant who responds in a playful manner when asked an irrelevant question."""
    
    # --- FIX: Use a placeholder for consistency ---
    human_message = "I am an expert on ocean data. While that is a fascinating question, it is a bit outside my area of expertise. How about we check the sea surface temperature somewhere?"
    funny_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input_text}")])
    
    funny_llm = ChatOllama(model="phi3", temperature=0.7)
    funny_response = funny_prompt | funny_llm | StrOutputParser()
    message = funny_response.invoke({"input_text": human_message})
    state["query_result"] = message
    print("Generated funny response.")
    return state

def end_max_iterations(state: AgentState):
    state["query_result"] = "I am having trouble formulating a query for your request. Please try rephrasing."
    print("Maximum attempts reached. Ending the workflow.")
    return state

def relevance_router(state: AgentState):
    if state["relevance"].lower() == "relevant":
        return "convert_to_sql"
    else:
        return "generate_funny_response"

def check_attempts_router(state: AgentState):
    if state["attempts"] < 3:
        return "convert_to_sql"
    else:
        return "end_max_iterations"

def execute_sql_router(state: AgentState):
    if not state.get("sql_error", False):
        return "generate_human_readable_answer"
    else:
        return "regenerate_query"

# --- Graph Definition ---
workflow = StateGraph(AgentState)
workflow.add_node("check_relevance", check_relevance)
workflow.add_node("convert_to_sql", convert_nl_to_sql)
workflow.add_node("execute_sql", execute_sql)
workflow.add_node("generate_human_readable_answer", generate_human_readable_answer)
workflow.add_node("regenerate_query", regenerate_query)
workflow.add_node("generate_funny_response", generate_funny_response)
workflow.add_node("end_max_iterations", end_max_iterations)

workflow.set_entry_point("check_relevance")
workflow.add_conditional_edges(
    "check_relevance",
    relevance_router,
    {"convert_to_sql": "convert_to_sql", "generate_funny_response": "generate_funny_response"},
)
workflow.add_edge("convert_to_sql", "execute_sql")
workflow.add_conditional_edges(
    "execute_sql",
    execute_sql_router,
    {"generate_human_readable_answer": "generate_human_readable_answer", "regenerate_query": "regenerate_query"},
)
workflow.add_conditional_edges(
    "regenerate_query",
    check_attempts_router,
    {"convert_to_sql": "convert_to_sql", "end_max_iterations": "end_max_iterations"},
)
workflow.add_edge("generate_human_readable_answer", END)
workflow.add_edge("generate_funny_response", END)
workflow.add_edge("end_max_iterations", END)

app = workflow.compile()

# --- Example of how to run the agent ---
if __name__ == "__main__":
    print("SQL Agent for Ocean Data is ready. Ask a question.")
    
    # Example question
    user_question = "what is the sea water potential temperature and salinity at longitude 75 and latitude 5 at a depth of 5 meters?"
    
    inputs = {"question": user_question, "attempts": 0}
    
    final_answer = None
    
    print("\n--- Agent Execution Log ---")
    for event in app.stream(inputs):
        for key, value in event.items():
            print(f"--- Event: {key} ---")
            print(value)
            # Capture the final answer when the last node completes
            if key == "generate_human_readable_answer":
                final_answer = value.get("query_result")
    
    print("\n" + "="*50)
    print("✅ Final Human-Readable Answer:")
    print("="*50)
    if final_answer:
        print(final_answer)
    else:
        print("Could not generate a final answer from the workflow.")



SQL Agent for Ocean Data is ready. Ask a question.

--- Agent Execution Log ---
Retrieved database schema.
Checking relevance of the question: what is the sea water potential temperature and salinity at longitude 75 and latitude 5 at a depth of 5 meters?
Relevance determined: relevant
--- Event: check_relevance ---
{'question': 'what is the sea water potential temperature and salinity at longitude 75 and latitude 5 at a depth of 5 meters?', 'attempts': 0, 'relevance': 'relevant'}
Retrieved database schema.
Converting question to SQL: what is the sea water potential temperature and salinity at longitude 75 and latitude 5 at a depth of 5 meters?
Generated SQL query: SELECT so, thetao FROM ocean_data WHERE (latitude - 5.0)*(latitude - 5.0) + (longitude - 75.0)*(longitude - 75.0) + (depth - 5.0)*(depth - 5.0) ASC LIMIT 1;
--- Event: convert_to_sql ---
{'question': 'what is the sea water potential temperature and salinity at longitude 75 and latitude 5 at a depth of 5 meters?', 'attempts': 